# Movie Analysis Project by Brooke Smyth, Garrett Williams, and Scott Schumann

**Overview:** [LET'S FILL IN A ONE PARAGRAPH OVERVIEW OF THE PROJECT LATER]

**Business Problem:** [SUMMARY OF BUSINESS PROBLEM TO BE FILLED IN LATER]

**Data Understanding:** [DESCRIPTION OF DATA BEING USED IN THIS PROJECT TO BE FILLED IN LATER]

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

**Data Preparation:** [DESCRIBE AND JUSTIFY THE PROCESS OF PREPARING DATA FOR ANALYSIS TO BE FILLED IN LATER]

In [2]:
imdb_name = pd.read_csv('zippedData/imdb.name.basics.csv.gz')
imdb_title = pd.read_csv('zippedData/imdb.title.basics.csv.gz')
imdb_crew = pd.read_csv('zippedData/imdb.title.crew.csv.gz')
imdb_principals = pd.read_csv('zippedData/imdb.title.principals.csv.gz')
imdb_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz')
rt_info = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter="\t")
rt_reviews = pd.read_csv('zippedData/rt.reviews.tsv.gz', sep="\t", encoding='latin-1')
tn = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

In [3]:
# Merging dataframes
tn.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [4]:
#create a new column that has the year as an integer
tn['year_int'] = tn['release_date'].str[-4:].astype(int)

#get info from movies since 2010 because that's as far back as the imdb data goes
#tn = tn[tn['year_int'] >= 2010]

#convert the values in the worldwide gross column to integers
tn["ww_gross_int"] = tn["worldwide_gross"].replace("[\$,]", "", regex=True).astype(int)

#convert the values in the production budget column to integers
tn["pro_budget_int"] = tn["production_budget"].replace("[\$,]", "", regex=True).astype(int)

#create a new column that calculates return on investment 
tn["roi"] = (tn["ww_gross_int"] - tn["pro_budget_int"])/tn["pro_budget_int"]

#sort the movies by return on investment
tn.sort_values(by = "roi", ascending = False, inplace = True)

In [5]:
imdb_title_rb = pd.merge(imdb_title, imdb_ratings, on = "tconst", how = "left")

In [6]:
imdb_title_crb = pd.merge(imdb_title_rb, imdb_crew, on = "tconst", how = "left")
imdb_title_crb.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,directors,writers
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,nm0712540,"nm0023551,nm1194313,nm0347899,nm1391276"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,nm0002411,NaN
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,nm0000080,"nm0000080,nm0462648"
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0,nm0611531,nm0347899
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,"nm0765384,nm0749914","nm1360635,nm0749914"


In [7]:
imdb_merged = pd.merge(imdb_title_crb, imdb_name, left_on = "directors", right_on = "nconst", how = "left")
imdb_merged.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,directors,writers,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,nm0712540,"nm0023551,nm1194313,nm0347899,nm1391276",nm0712540,Harnam Singh Rawail,1921.0,2004.0,"director,writer,producer","tt0266757,tt0266712,tt0134996,tt0266765"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,nm0002411,NaN,nm0002411,Mani Kaul,1944.0,2011.0,"director,writer,actor","tt0102515,tt0070009,tt0066514,tt0207626"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,nm0000080,"nm0000080,nm0462648",nm0000080,Orson Welles,1915.0,1985.0,"actor,director,writer","tt0048393,tt0052311,tt0057427,tt0033467"
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0,nm0611531,nm0347899,nm0611531,Hrishikesh Mukherjee,1922.0,2006.0,"director,editor,writer","tt0066763,tt0147855,tt0137362,tt0137361"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,"nm0765384,nm0749914","nm1360635,nm0749914",NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#Merging the TN and IMDB dataframes, which only includes data from 2010 to the present. We feel this is 
#relevant because the stakeholder is trying to start a new studio and thus more recent data is probably
#more useful.
tn_imdb_merged = pd.merge(tn, imdb_merged, left_on = ['movie', 'year_int'], right_on = ['primary_title', 'start_year'], how = 'inner')
tn_imdb_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1547 entries, 0 to 1546
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  1547 non-null   int64  
 1   release_date        1547 non-null   object 
 2   movie               1547 non-null   object 
 3   production_budget   1547 non-null   object 
 4   domestic_gross      1547 non-null   object 
 5   worldwide_gross     1547 non-null   object 
 6   year_int            1547 non-null   int64  
 7   ww_gross_int        1547 non-null   int64  
 8   pro_budget_int      1547 non-null   int64  
 9   roi                 1547 non-null   float64
 10  tconst              1547 non-null   object 
 11  primary_title       1547 non-null   object 
 12  original_title      1547 non-null   object 
 13  start_year          1547 non-null   int64  
 14  runtime_minutes     1521 non-null   float64
 15  genres              1541 non-null   object 
 16  averag

**Data Modeling:**